In [31]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [32]:
path = '/Users/kevinpan/Downloads/NFL Salary By Position Group.xlsx - Sheet1.csv'
data = pd.read_csv(path)

In [33]:
data

,Team_Season,Team,QB,RB,WR,TE,OL,Offense,IDL,EDGE,...,OL_P,IDL_P,EDGE_P,LB_P,S_P,CB_P,Defense_P,Offense_P,Dead_Open_Specials,W
0,Cardinals 2022,Cardinals,16348925,6213758,24621833,7121287,44681251,98987054,23503243,9842220,...,0.214607,0.112888,0.047273,0.073820,0.094174,0.034218,0.362372,0.475442,0.162186,4
1,Falcons 2022,Falcons,6303695,4894510,9535858,10678128,32848444,64260635,18104464,6077102,...,0.157774,0.086957,0.029189,0.023618,0.021461,0.067402,0.228627,0.308649,0.462725,7
2,Ravens 2022,Ravens,24100655,8179654,7616822,12307075,38678276,90882482,11195023,14877531,...,0.185775,0.053771,0.071458,0.037714,0.057958,0.154194,0.375094,0.436515,0.188390,10
3,Bills 2022,Bills,20230481,7299886,18982654,4501683,29350967,80365671,16458562,12078022,...,0.140975,0.079052,0.058012,0.115715,0.087184,0.126374,0.466336,0.386002,0.147662,13
4,Panthers 2022,Panthers,20461726,3829960,12882679,5253310,33291588,75719263,11405978,9462303,...,0.159902,0.054784,0.045448,0.111972,0.032674,0.080464,0.325342,0.363685,0.310973,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Patriots 2013,Patriots,14603953,3682367,8749814,4150882,22775490,53962506,15169500,6420702,...,0.184268,0.122731,0.051947,0.087388,0.059763,0.069731,0.391560,0.436590,0.171850,12
316,Packers 2013,Packers,13013000,2429420,10223980,11341695,17211503,54219598,15831195,12650974,...,0.139252,0.128084,0.102354,0.073631,0.040204,0.112922,0.457196,0.438670,0.104135,8
317,Steelers 2013,Steelers,15393138,2549641,8799844,9207474,10836255,46786352,12586770,12823812,...,0.087672,0.101835,0.103753,0.074072,0.133337,0.089562,0.502558,0.378530,0.118912,8
318,Commanders 2013,Commanders,5972967,2212882,19735321,4645357,20025614,52592141,14227749,9152416,...,0.162020,0.115111,0.074049,0.084763,0.037376,0.057463,0.368762,0.425503,0.205735,3


In [34]:
features = ['QB', 'RB', 'WR', 'TE', 'OL', 'IDL', 'EDGE', 'LB', 'S', 'CB']
target = 'W'

X = data[features].values
y = data[target].values

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)

In [38]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [39]:
class winNN(nn.Module):
    def __init__(self, input_size):
        super(winNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.fc(x)

In [40]:
model = winNN(input_size=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [41]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/1000], Loss: 9.5862
Epoch [20/1000], Loss: 8.3049
Epoch [30/1000], Loss: 6.8344
Epoch [40/1000], Loss: 10.2125
Epoch [50/1000], Loss: 4.4997
Epoch [60/1000], Loss: 6.1610
Epoch [70/1000], Loss: 8.0427
Epoch [80/1000], Loss: 5.0821
Epoch [90/1000], Loss: 7.4878
Epoch [100/1000], Loss: 6.3482
Epoch [110/1000], Loss: 8.1842
Epoch [120/1000], Loss: 5.0096
Epoch [130/1000], Loss: 6.3982
Epoch [140/1000], Loss: 4.0143
Epoch [150/1000], Loss: 5.0836
Epoch [160/1000], Loss: 3.3202
Epoch [170/1000], Loss: 6.3451
Epoch [180/1000], Loss: 4.6359
Epoch [190/1000], Loss: 3.4800
Epoch [200/1000], Loss: 3.9080
Epoch [210/1000], Loss: 5.0066
Epoch [220/1000], Loss: 2.6172
Epoch [230/1000], Loss: 3.6028
Epoch [240/1000], Loss: 3.6234
Epoch [250/1000], Loss: 1.2846
Epoch [260/1000], Loss: 1.9023
Epoch [270/1000], Loss: 1.3765
Epoch [280/1000], Loss: 1.3386
Epoch [290/1000], Loss: 0.8727
Epoch [300/1000], Loss: 1.0324
Epoch [310/1000], Loss: 1.0613
Epoch [320/1000], Loss: 1.2736
Epoch [330/1000]

In [42]:
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.numpy()

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 27.625280492164652
R-squared Score: -1.2051982879638672


In [43]:
X_test_tensor

tensor([[-1.5874, -0.7183, -1.5905, -1.1117,  1.7564, -1.2513, -0.9645,  1.9401,
         -1.2223, -1.0461],
        [-0.6596, -0.8417,  0.3038, -0.9311, -0.0745, -0.1111, -0.6417,  1.7476,
         -0.3564,  0.2617],
        [-0.6574, -0.3717, -0.1872, -1.0847, -1.0153,  2.7532,  1.4051,  1.0205,
          0.7005,  0.1586],
        [ 1.6912, -0.1085,  1.2491, -0.3344, -0.1191, -0.0118,  1.7572, -1.4574,
          0.4877, -0.5945],
        [ 0.2468, -0.1628,  0.4032, -0.5992,  1.9279, -0.1624, -0.7830, -0.8351,
          2.7361,  1.0787],
        [-0.7694, -0.0616,  1.3150,  1.2980,  0.4607, -0.9304, -0.3626,  0.1152,
          0.9039, -0.7568],
        [-1.6949, -1.1905, -0.3823, -0.9644, -0.1259, -0.8177,  0.1421, -0.4637,
         -0.0669, -1.1440],
        [ 1.1520,  0.2072,  0.2935,  1.0022,  1.1404, -0.4148, -1.8295, -0.2245,
          0.2465, -0.4592],
        [-0.1334, -0.7557, -0.0723,  1.1384, -0.6478, -0.9332, -0.5048,  0.2114,
          0.3288, -1.4023],
        [-0.4752, -

In [44]:
y_pred

array([[ 3.2254293],
       [ 3.1337504],
       [16.199408 ],
       [11.533545 ],
       [ 9.539642 ],
       [10.590984 ],
       [ 2.1679642],
       [ 6.5863543],
       [11.52257  ],
       [ 7.4011507],
       [ 8.5560465],
       [14.897154 ],
       [ 5.549652 ],
       [ 1.5610191],
       [ 8.47303  ],
       [10.810502 ],
       [ 2.7703545],
       [ 3.2336862],
       [ 5.4984417],
       [11.924923 ],
       [11.805782 ],
       [ 7.6510305],
       [ 7.515179 ],
       [ 5.913675 ],
       [ 2.062288 ],
       [ 9.603732 ],
       [ 8.287018 ],
       [ 8.174935 ],
       [20.96541  ],
       [ 9.121309 ],
       [ 5.155081 ],
       [15.469261 ],
       [19.462183 ],
       [ 8.115011 ],
       [10.844519 ],
       [19.141743 ],
       [ 3.4421349],
       [ 9.515569 ],
       [10.584112 ],
       [13.619328 ],
       [ 9.368626 ],
       [10.541995 ],
       [10.260795 ],
       [ 8.124083 ],
       [16.883104 ],
       [ 7.0870996],
       [ 8.688934 ],
       [11.31

In [45]:
def infer(model, input_data, scaler):
    model.eval()
    input_data_scaled = scaler.transform([input_data])  # Scale the input data
    input_tensor = torch.FloatTensor(input_data_scaled).unsqueeze(0)  # Ensure input is 2D
    with torch.no_grad():
        prediction = model(input_tensor)
    return prediction.item()

sample_input = [16348925, 6213758, 24621833, 7121287, 44681251, 23503243, 9842220, 15369345, 19606956, 7124107]  # Replace with actual values
predicted_wins = infer(model, sample_input, scaler)
print("Predicted Wins:", predicted_wins)

Predicted Wins: 3.7894818782806396


In [43]:
torch.save(model, 'nfl_wins_spending_nn_simple.pth')

In [46]:
import joblib

joblib.dump(scaler, 'nfl_win_scaler.pkl')

['nfl_win_scaler.pkl']